In [ ]:
import pandas as pd
import statsmodels.api as sts

In [ ]:
mynorm = pd.read_parquet("../data/processed/CorrectedMyNorms/mynorm.parquet")
samplesheet = pd.read_csv("../data/raw/SampleSheet.csv", index_col=0)

In [ ]:
probes = pd.read_csv("../Files/COVSpecificDMPs.csv", index_col=0)
mynorm = mynorm.T[probes.index]

In [ ]:
samplesheet.Sex = samplesheet.Sex.replace({"F": 1, "M": 0})
samplesheet.Status = samplesheet.Status.replace(
    {
        "COVID-19 PL": "COVID",
        "COVID-19 ES": "COVID",
        "COVID-19 USA 1": "COVID",
        "COVID-19 USA 2": "COVID",
        "Other respiratory infections USA 1": "non-COVID",
        "Other respiratory infections USA 2": "non-COVID",
        "Healthy controls": "Control",
    }
)
samplesheet["intercept"] = 1

In [ ]:
pd.crosstab(samplesheet.Status, samplesheet.Sex)

In [ ]:
for probe in probes.index:
    model = sts.MNLogit(
        endog=samplesheet.Status.loc[mynorm.index],
        exog=pd.concat(
            (mynorm[probe], samplesheet[["intercept", "Sex", "Age"]]), axis=1
        ),
    )
    summary = model.fit()
    print(summary.summary())
    assert all(summary.pvalues.loc[probe, :] <= 0.05)